In [14]:
import heapq
import geopy.distance
import math
import csv
import Utils
import time
  
PUBLIC_TRANSPORT_SPEED_KM_H = 10

class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append((end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
            else:
                self.graph_dict[start] = [(end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon)]
            if end not in self.graph_dict:
                self.graph_dict[end] = []
                
def edges(fileName):
    edges = []
    with open(fileName, 'r', encoding='utf8') as file:
        reader = csv.reader(file)
        header = next(reader)
        
        for row in reader:
            line = row[2]
            start_time = Utils.minutes_from_midnight(row[3])
            arrival_time = Utils.minutes_from_midnight(row[4]) 
            weight = arrival_time - start_time
            
            if weight < 0:
                weight += 24*60
            
            start = row[5]
            end = row[6]
            
            start_stop_lat = row[7]
            start_stop_lon = row[8]
            end_stop_lat = row[9]
            end_stop_lon = row[10]
            
            edges.append((start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
    return edges

In [15]:
def distance_from_coords(coords1, coords2):
    return geopy.distance.geodesic(coords1, coords2).km

def distance_from_coords_manhatann(coords1, coords2):
        coords1_lat, coords1_lon = coords1
        coords2_lat, coords2_lon = coords2
        return geopy.distance.geodesic((coords1_lat, coords1_lon), (coords2_lat, coords1_lon)).km + geopy.distance.geodesic((coords1_lat, coords1_lon), (coords1_lat, coords2_lon)).km

def time_from_distance(distance):
    return distance/PUBLIC_TRANSPORT_SPEED_KM_H * 60
    
def time_from_coords(coords1, coords2):
    return time_from_distance(distance_from_coords(coords1, coords2))


In [16]:
def astar_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    goal_info = graph_dict[goal][0]
    coords2 = (goal_info[5], goal_info[6])
    counter = 0
    pq = [(0, start)]
    prev_nodes = {start: None}
    
    distances = {node: float('inf') for node in graph_dict}
    distances[start] = 0
    
    fScore = {node: float('inf') for node in graph_dict}
    fScore[start] = 0
    
    trip_start_time = Utils.minutes_from_midnight(trip_start_time)
    while pq:
        curr_f_score, current = heapq.heappop(pq)
        curr_cost = distances[current]
        curr_arrival_time = trip_start_time + curr_cost
        base_days_en_route = 0
        if current == goal:
            break
        
        # if curr_f_score > fScore[current]:
        #     continue
        
        if curr_arrival_time > 24*60:
            base_days_en_route = int(curr_arrival_time/(24*60))
            curr_arrival_time = curr_arrival_time%(24*60)
            
        prev_line = ''
        if prev_nodes[current]:
            prev_line = prev_nodes[current][1]
            
        for neighbor, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon in graph_dict[current]:
            counter += 1
            days_en_route = base_days_en_route
            
            coords1 = (start_stop_lat, start_stop_lon)

            if curr_arrival_time <= start_time:
                # current travel time + time spent waiting for the bus/tram + time spent en route 
                new_cost = curr_cost + (start_time - (curr_arrival_time)) + weight
            else:
                # current travel time + (time till midnight + start_time) + time spent en route
                new_cost = curr_cost + (24*60 - (curr_arrival_time) + start_time) + weight
                days_en_route += 1
            
            if new_cost < distances[neighbor]:
                distances[neighbor] = new_cost
                priority = new_cost + math.floor(coords_heuristic_fn(coords1, coords2))
                fScore[neighbor] = priority
                heapq.heappush(pq, (priority, neighbor))
                prev_nodes[neighbor] = (current, line, start_time + days_en_route*24*60, arrival_time + days_en_route*24*60)

    # print(f'COUNTER: {counter}')
    return distances, prev_nodes


In [17]:
def shortest_path_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    distances, prev_nodes = astar_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)

In [18]:
gg = Graph(edges('connection_graph_fixed.csv'))

In [19]:
# shortest_path_t(gg.graph_dict, 'pl. Bema', 'DWORZEC GŁÓWNY', '12:00:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Ogród Botaniczny', 'Rynek', '20:00:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Białowieska', 'most Grunwaldzki', '12:00:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Kwiska', 'PL. GRUNWALDZKI', '09:00:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Iwiny - rondo', 'Hala Stulecia', '14:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Krzemieniecka', 'Prusa', '20:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'KOSZAROWA (Szpital)', 'Waniliowa', '07:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Ossolineum (Uniwersytecka)', 'Kasprowicza', '23:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Na Ostatnim Groszu', 'Biegasa', '9:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Orla', 'pl. Strzegomski (Muzeum Współczesne)', '21:07:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Damrota', 'GIEŁDOWA (Centrum Hurtu)', '00:12:00', time_from_coords)
# shortest_path_t(gg.graph_dict, 'Łozina - Wrocławska (na wys. nr 18)', 'Chopina', '10:07:00', time_from_coords)
shortest_path_t(gg.graph_dict, 'Magellana', 'DWORZEC AUTOBUSOWY', '10:07:00', time_from_coords)

time_taken: 0.086s
time: 00:30:00
start time: 10:07:00
in: Magellana [10:08:00]| line: 941
out: SĘPOLNO [10:12:00]| line: 941
in: SĘPOLNO [10:13:00]| line: 9
out: Kochanowskiego [10:20:00]| line: 9
in: Kochanowskiego [10:21:00]| line: 12
out: GALERIA DOMINIKAŃSKA [10:31:00]| line: 12
in: GALERIA DOMINIKAŃSKA [10:31:00]| line: 2
out: DWORZEC GŁÓWNY [10:34:00]| line: 2
in: DWORZEC GŁÓWNY [10:34:00]| line: K
out: DWORZEC AUTOBUSOWY [10:37:00]| line: K
-
-
-


In [20]:
def shortest_path_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    distances, prev_nodes = astar_t(graph_dict, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)
    return Utils.minutes_from_midnight(trip_start_time) + trip_length

def printTabu(stops, startTime, grapth_dict):
    start_time = startTime
    for i in range(stops.__len__()):
        start_time = Utils.to_time(shortest_path_t(grapth_dict, stops[i], stops[(i+1)%stops.__len__()], start_time, distance_from_coords))

In [21]:
import random
import math

def Tabu(stops, trip_start_time, gg):
    calc_start_time = time.time()
    n_stops = len(stops)
    max_iterations = math.ceil(1.1*(n_stops**2))
    turns_improved = 0
    improve_thresh=2*math.floor(math.sqrt(max_iterations))
    tabu_list = []
    tabu_tenure = n_stops
    # tabu_tenure = 100000000

    distances = [[astar_t(gg.graph_dict, city1, city2, trip_start_time, distance_from_coords)[0][city2] for city1 in stops] for city2 in stops]

    total=0
    for i in range(n_stops):
            for j in range(n_stops):
                total += distances[i][j]

    aspiration_criteria = (total/(n_stops**2))*2.2
    # aspiration_criteria = -1

    current_solution = list(range(n_stops))
    best_solution = current_solution[:]
    best_solution_cost = sum([distances[current_solution[i]][current_solution[(i+1)%n_stops]] for i in range(n_stops)])

    for iteration in range(max_iterations):
        if turns_improved>improve_thresh:
            break
        best_neighbor = None
        best_neighbor_cost = float('inf')
        tabu_candidate = (0,0)
        for i in range(n_stops):
            for j in range(i+1, n_stops):
                neighbor = current_solution[:]
                if i > 0:
                    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbor_cost = sum([distances[neighbor[i]][neighbor[(i+1)%n_stops]] for i in range(n_stops)])
                if (i,j) not in tabu_list or neighbor_cost < aspiration_criteria:
                    if neighbor_cost < best_neighbor_cost:
                        best_neighbor = neighbor[:]
                        best_neighbor_cost = neighbor_cost
                        tabu_candidate = (i,j)
        if best_neighbor is not None:
            current_solution = best_neighbor[:]
            tabu_list.append(tabu_candidate)
            if len(tabu_list) > tabu_tenure:
                tabu_list.pop(0)
            if best_neighbor_cost < best_solution_cost:
                best_solution = best_neighbor[:]
                best_solution_cost = best_neighbor_cost
                turns_improved=0
            else:
                turns_improved=turns_improved+1

        # print("Iteration {}: Best solution cost = {}".format(iteration, best_solution_cost))

    calc_end_time = time.time()
    print(f"time: {calc_end_time - calc_start_time}")
    stops_sorted = [stops[i] for i in best_solution]
    
    printTabu(stops_sorted, trip_start_time, gg.graph_dict)
    
    print(f"Best solution: {stops_sorted}")
    # print("Best solution cost: {}".format(best_solution_cost))
    

In [22]:
stops1 = ['Kwiska', 'PL. GRUNWALDZKI', 'Ogród Botaniczny', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
stops2 = ['pl. Bema', 'Łozina - Wrocławska (na wys. nr 18)', 'Kwiska', 'DWORZEC GŁÓWNY', 'GIEŁDOWA (Centrum Hurtu)']
stops3 = ['pl. Bema', 'PL. GRUNWALDZKI', 'Kwiska']
stops4 = ['DWORZEC GŁÓWNY', 'Łozina - Wrocławska (na wys. nr 18)', 'GIEŁDOWA (Centrum Hurtu)']

trip_start_time = "12:00:00"
Tabu(stops1, trip_start_time, gg)
Tabu(stops2, trip_start_time, gg)
Tabu(stops3, trip_start_time, gg)
Tabu(stops4, trip_start_time, gg)

time: 8.623200416564941
time_taken: 0.496s
time: 00:24:00
start time: 12:00:00
in: Kwiska [12:00:00]| line: 3
out: PL. JANA PAWŁA II [12:10:00]| line: 3
in: PL. JANA PAWŁA II [12:10:00]| line: 14
out: Dubois [12:17:00]| line: 14
in: Dubois [12:17:00]| line: 6
out: pl. Bema [12:20:00]| line: 6
in: pl. Bema [12:22:00]| line: 9
out: Ogród Botaniczny [12:24:00]| line: 9
-
-
-
time_taken: 0.042s
time: 00:07:00
start time: 12:24:00
in: Ogród Botaniczny [12:24:00]| line: 9
out: Piastowska [12:26:00]| line: 9
in: Piastowska [12:28:00]| line: 1
out: PL. GRUNWALDZKI [12:31:00]| line: 1
-
-
-
time_taken: 0.241s
time: 00:15:00
start time: 12:31:00
in: PL. GRUNWALDZKI [12:32:00]| line: 16
out: most Grunwaldzki [12:33:00]| line: 16
in: most Grunwaldzki [12:34:00]| line: D
out: GALERIA DOMINIKAŃSKA [12:39:00]| line: D
in: GALERIA DOMINIKAŃSKA [12:42:00]| line: N
out: DWORZEC GŁÓWNY [12:46:00]| line: N
-
-
-
time_taken: 0.916s
time: 00:31:00
start time: 12:46:00
in: DWORZEC GŁÓWNY [12:48:00]| line: 19